In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

import pandas as pd
import numpy as np
import os


In [3]:
TRAIN_CSV = "../data/clean/train_cleaned.csv"
TEST_CSV = "../data/clean/test_cleaned.csv"
IMAGE_DIR = "../data/raw/food-101/images/"


In [4]:
train_df = pd.read_csv(TRAIN_CSV)[["image_name", "label"]]
test_df = pd.read_csv(TEST_CSV)[["image_name", "label"]]

print(train_df.head())
print(test_df.head())


        image_name    label
0  churros/1004234  churros
1  churros/1004234  churros
2  churros/1013460  churros
3  churros/1013460  churros
4  churros/1016791  churros
        image_name    label
0  churros/1061830  churros
1  churros/1064042  churros
2  churros/1074903  churros
3  churros/1085259  churros
4  churros/1097261  churros


In [5]:
def make_path(x):
    return os.path.join(IMAGE_DIR, x + ".jpg")

train_df["image_path"] = train_df["image_name"].apply(make_path)
test_df["image_path"] = test_df["image_name"].apply(make_path)


In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
train_df["label_encoded"] = label_encoder.fit_transform(train_df["label"])
test_df["label_encoded"] = label_encoder.transform(test_df["label"])

num_classes = len(label_encoder.classes_)
print("Classes:", num_classes)


Classes: 101


In [7]:
import json
with open("../data/clean/label_map.json", "w") as f:
    json.dump({c: int(i) for i, c in enumerate(label_encoder.classes_)}, f)


In [8]:
IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="image_path",
    y_col="label_encoded",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="raw",     
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    test_df,
    x_col="image_path",
    y_col="label_encoded",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="raw",     
    shuffle=False
)


Found 83250 validated image filenames.
Found 25250 validated image filenames.


In [9]:
base = ResNet50(weights="imagenet", include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

x = base.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
preds = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base.input, outputs=preds)

In [10]:
for layer in base.layers:
    layer.trainable = False


In [11]:
model.compile(
    optimizer=Adam(1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [12]:
checkpoint = ModelCheckpoint(
    "../data/clean/resnet50_food101_head.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    patience=2,
    factor=0.2,
    min_lr=1e-6,
    verbose=1
)

early = EarlyStopping(
    patience=4,
    restore_best_weights=True,
    verbose=1
)

csv_logger = CSVLogger("../data/clean/resnet50_training_log.csv")


In [14]:
history_head = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=8,
    callbacks=[checkpoint, reduce_lr, early, csv_logger]
)


Epoch 1/8
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.2577 - loss: 3.1387
Epoch 1: val_accuracy improved from -inf to 0.53493, saving model to ../data/clean/resnet50_food101_head.keras
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 4218s 2s/step - accuracy: 0.2577 - loss: 3.1385 - val_accuracy: 0.5349 - val_loss: 1.7740 - learning_rate: 0.0010
Epoch 2/8
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3857 - loss: 2.4728
Epoch 2: val_accuracy improved from 0.53493 to 0.55085, saving model to ../data/clean/resnet50_food101_head.keras
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 4018s 2s/step - accuracy: 0.3857 - loss: 2.4728 - val_accuracy: 0.5509 - val_loss: 1.7155 - learning_rate: 0.0010
Epoch 3/8
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4067 - loss: 2.3649
Epoch 3: val_accuracy improved from 0.55085 to 0.56131, saving model to ../data/clean/resnet50_food101_head.keras
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 4094s 2s/step - accuracy: 0.4067 - loss: 2.3649 - val_accuracy: 0.5613 - val_loss:

In [15]:
for layer in base.layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [16]:
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    callbacks=[checkpoint, reduce_lr, early, csv_logger]
)

model.save("../data/clean/resnet50_food101_final.keras")


Epoch 1/10
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.4288 - loss: 2.3255
Epoch 1: val_accuracy improved from 0.59564 to 0.64412, saving model to ../data/clean/resnet50_food101_head.keras
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 6136s 2s/step - accuracy: 0.4288 - loss: 2.3254 - val_accuracy: 0.6441 - val_loss: 1.3493 - learning_rate: 1.0000e-05
Epoch 2/10
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5308 - loss: 1.8298
Epoch 2: val_accuracy improved from 0.64412 to 0.67184, saving model to ../data/clean/resnet50_food101_head.keras
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 6385s 2s/step - accuracy: 0.5308 - loss: 1.8298 - val_accuracy: 0.6718 - val_loss: 1.2368 - learning_rate: 1.0000e-05
Epoch 3/10
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5660 - loss: 1.6730
Epoch 3: val_accuracy improved from 0.67184 to 0.69263, saving model to ../data/clean/resnet50_food101_head.keras
2602/2602 ━━━━━━━━━━━━━━━━━━━━ 6596s 3s/step - accuracy: 0.5660 - loss: 1.6730 - val_accuracy: 0.69

In [17]:
import tensorflow as tf
import numpy as np
import json
import pandas as pd
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image

model = tf.keras.models.load_model("../data/clean/resnet50_food101_final.keras")

with open("../data/clean/label_map.json") as f:
    label_map = json.load(f)

inv_map = {v: k for k, v in label_map.items()}


In [18]:
def predict_food(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_arr = preprocess_input(img_arr)

    preds = model.predict(img_arr)
    idx = np.argmax(preds)

    return inv_map[idx], preds[0][idx]


In [19]:
usda_df = pd.read_csv("../data/raw/usda_food_data.csv")

from rapidfuzz import process

def get_nutrition(food_label):
    descs = usda_df["description"].astype(str).tolist()
    match, score = process.extractOne(food_label.replace("_", " "), descs)
    row = usda_df[usda_df["description"] == match]
    if len(row):
        return row.iloc[0]
    return None


In [23]:
model.save("../data/clean/resnet50_food101_finetuned.keras")
print("Model saved in .keras format")

model.save("../data/clean/resnet50_food101_finetuned.h5")
print("Model also saved in .h5 format")

model.save_weights("../data/clean/resnet50_food101.weights.h5")
print("Model weights saved separately")

Model saved in .keras format
Model also saved in .h5 format
Model weights saved separately
